### Import Block

In [1]:
import os
import glob
import numpy as np
import nibabel as nib
from pathlib import Path

In [2]:
import tensorflow as tf

In [3]:
from google.colab import files

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing Block

##### Path and constants

In [ ]:
#!unzip "/content/drive/MyDrive/Colab_Notebooks/segmentation-model-1/brats2023_preprocessed.zip" -d "/content/"
!unzip "/content/drive/MyDrive/Colab/Copie de brats2023_preprocessed.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Colab/Copie de brats2023_preprocessed.zip
   creating: /content/brats2023_preprocessed/
  inflating: /content/__MACOSX/._brats2023_preprocessed  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-00823-000.npz  
  inflating: /content/__MACOSX/brats2023_preprocessed/._BraTS-GLI-00823-000.npz  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-00036-001.npz  
  inflating: /content/__MACOSX/brats2023_preprocessed/._BraTS-GLI-00036-001.npz  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-00687-000.npz  
  inflating: /content/__MACOSX/brats2023_preprocessed/._BraTS-GLI-00687-000.npz  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-00697-000.npz  
  inflating: /content/__MACOSX/brats2023_preprocessed/._BraTS-GLI-00697-000.npz  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-00499-000.npz  
  inflating: /content/__MACOSX/brats2023_preprocessed/._BraTS-GLI-00499-000.npz  
  inflating: /content/brats2023_preprocessed/BraTS-GLI-0

In [ ]:
from pathlib import Path

PREPROC_ROOT = Path("/content/brats2023_preprocessed")



all_npz = sorted(str(p) for p in PREPROC_ROOT.glob("*.npz"))
print("Nb de fichiers .npz :", len(all_npz))


PATCH_SIZE = (128, 128, 128)
BATCH_SIZE = 2
AUTOTUNE = tf.data.AUTOTUNE

# RAW_ROOT = Path("../../../../raw_data/segmentation/brats2023_raw")

# Taille cible
TARGET_SHAPE = (160, 192, 160)  # (H, W, D)

# On choisit un ordre de canaux cohérent (à garder ensuite dans le Model)
# [T1 = t1 native, T1c, T2w, T2 FLAIR]
MODALITIES = [
    ("t1n", "*-t1n.nii.gz"),
    ("t1c", "*-t1c.nii.gz"),
    ("t2w", "*-t2w.nii.gz"),
    ("t2f", "*-t2f.nii.gz"),
]

Nb de fichiers .npz : 1251


##### Loading function for .npz files

In [ ]:
def load_npz(path):
    path = path.decode("utf-8")  # tf.string -> str
    data = np.load(path)
    img = data["image"]  # (H,W,D,4)
    seg = data["label"]  # (H,W,D)
    # On ajoute éventuellement un canal au label pour être (H,W,D,1)
    seg = np.expand_dims(seg, axis=-1)  # (H,W,D,1)
    return img.astype(np.float32), seg.astype(np.uint8)

def tf_load_npz(path):
    img, seg = tf.numpy_function(load_npz, [path], [tf.float32, tf.uint8])
    # Fixer les shapes statiques
    img.set_shape((*TARGET_SHAPE, len(MODALITIES)))  # (H,W,D,4)
    seg.set_shape((*TARGET_SHAPE, 1))                # (H,W,D,1)
    return img, seg

##### Extraction de patchs 3D aléatoires

In [ ]:

PATCH_H, PATCH_W, PATCH_D = PATCH_SIZE

def random_patch_3d(img, seg):
    """img: (H,W,D,4), seg: (H,W,D,1)"""
    shape = tf.shape(img)
    H, W, D = shape[0], shape[1], shape[2]

    max_h = H - PATCH_H
    max_w = W - PATCH_W
    max_d = D - PATCH_D

    # Sécurité si patch > volume
    max_h = tf.maximum(max_h, 0)
    max_w = tf.maximum(max_w, 0)
    max_d = tf.maximum(max_d, 0)

    h = tf.random.uniform((), 0, max_h + 1, dtype=tf.int32)
    w = tf.random.uniform((), 0, max_w + 1, dtype=tf.int32)
    d = tf.random.uniform((), 0, max_d + 1, dtype=tf.int32)

    img_patch = img[h:h+PATCH_H, w:w+PATCH_W, d:d+PATCH_D, :]
    seg_patch = seg[h:h+PATCH_H, w:w+PATCH_W, d:d+PATCH_D, :]

    img_patch.set_shape((PATCH_H, PATCH_W, PATCH_D, len(MODALITIES)))
    seg_patch.set_shape((PATCH_H, PATCH_W, PATCH_D, 1))

    return img_patch, seg_patch

In [ ]:
##### Data augmentation simple
def augment(img, seg):
    # Flip aléatoire gauche-droite (axe W)
    if tf.random.uniform(()) > 0.5:
        img = tf.reverse(img, axis=[1])
        seg = tf.reverse(seg, axis=[1])

    # Flip aléatoire avant-arrière (axe H)
    if tf.random.uniform(()) > 0.5:
        img = tf.reverse(img, axis=[0])
        seg = tf.reverse(seg, axis=[0])

    # Flip aléatoire sur l'axe de profondeur (D)
    if tf.random.uniform(()) > 0.5:
        img = tf.reverse(img, axis=[2])
        seg = tf.reverse(seg, axis=[2])

    # léger jitter d'intensité
    if tf.random.uniform(()) > 0.5:
        factor = tf.random.uniform((), 0.9, 1.1)
        img = img * factor

    return img, seg


##### Construction des datasets train / val

In [ ]:
# Liste de tous les fichiers .npz
from sklearn.model_selection import train_test_split

all_npz = sorted(str(p) for p in PREPROC_ROOT.glob("*.npz"))


train_paths, val_paths = train_test_split(all_npz, test_size=0.2, random_state=42)

def make_dataset(paths, augment_data=True):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(tf_load_npz, num_parallel_calls=AUTOTUNE)
    ds = ds.map(random_patch_3d, num_parallel_calls=AUTOTUNE)
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_paths, augment_data=True).repeat()
val_ds   = make_dataset(val_paths, augment_data=False)


##### Better loss function : Dice + CE

In [ ]:
def sparse_dice_loss(y_true, y_pred, smooth=1e-5):
    # y_true: (B,H,W,D,1) entiers [0..C-1]
    # y_pred: (B,H,W,D,C) probas softmax
    y_true = tf.squeeze(y_true, axis=-1)              # (B,H,W,D)
    y_true = tf.cast(y_true, tf.int32)                 # Cast to integer type
    y_true_one_hot = tf.one_hot(y_true, depth=tf.shape(y_pred)[-1])  # (B,H,W,D,C)

    # Dice par classe
    axes = (1,2,3)  # on somme sur H,W,D
    intersection = tf.reduce_sum(y_true_one_hot * y_pred, axis=axes)
    denom = tf.reduce_sum(y_true_one_hot + y_pred, axis=axes)

    dice = (2. * intersection + smooth) / (denom + smooth)
    dice_loss = 1 - dice  # shape (B,C)

    # moyenne des classes
    return tf.reduce_mean(dice_loss)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    ce = tf.reduce_mean(ce)
    dsc = sparse_dice_loss(y_true, y_pred)
    return ce + dsc

def dice_coef(y_true, y_pred, smooth=1e-6):
    # S'assurer qu'on est en float32
    y_true = tf.squeeze(y_true, axis=-1)              # (B,H,W,D)
    y_true = tf.cast(y_true, tf.int32)                 # Cast to integer type
    y_true_one_hot = tf.one_hot(y_true, depth=tf.shape(y_pred)[-1])  # (B,H,W,D,C)

    # Intersection et sommes sur toutes les dimensions sauf le batch
    # axes = [1, 2, 3, 4] pour (batch, H, W, D, C)
    axes = (1,2,3)  # on somme sur H,W,D
    intersection = tf.reduce_sum(y_true_one_hot * y_pred, axis=axes)
    denom = tf.reduce_sum(y_true_one_hot + y_pred, axis=axes)
    dice = (2. * intersection + smooth) / (denom + smooth)

    # Moyenne sur le batch
    return tf.reduce_mean(dice)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
def conv_block(x, filters):
    x = layers.Conv3D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x


def unet_3d(input_shape=(PATCH_H, PATCH_W, PATCH_D, 4), n_classes=4):

    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = conv_block(inputs, 32)
    p1 = layers.MaxPooling3D(2)(c1)

    c2 = conv_block(p1, 64)
    p2 = layers.MaxPooling3D(2)(c2)

    c3 = conv_block(p2, 128)
    p3 = layers.MaxPooling3D(2)(c3)

    # Bottleneck
    bn = conv_block(p3, 256)

    # Decoder
    u3 = layers.Conv3DTranspose(128, 2, strides=2, padding='same')(bn)
    u3 = layers.Concatenate()([u3, c3])
    c4 = conv_block(u3, 128)

    u2 = layers.Conv3DTranspose(64, 2, strides=2, padding='same')(c4)
    u2 = layers.Concatenate()([u2, c2])
    c5 = conv_block(u2, 64)

    u1 = layers.Conv3DTranspose(32, 2, strides=2, padding='same')(c5)
    u1 = layers.Concatenate()([u1, c1])
    c6 = conv_block(u1, 32)

    outputs = layers.Conv3D(n_classes, 1, activation="softmax", dtype='float32')(c6)
    return Model(inputs, outputs)


model_unet = unet_3d()
model_unet.summary()

model_unet.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=combined_loss,
    metrics=["accuracy", dice_coef])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_unet3d_v2.h5", save_best_only=True, monitor="val_loss"),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)]

# model_unet.compile(
#     optimizer="adam",
#     loss="sparse_categorical_crossentropy",
#     metrics=["accuracy"]
# )


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 128, 128,  │      3,488 │ input_layer[0][0] │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 128, 128,  │     27,680 │ activation[0][0]  │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv3d_1[0][0]    │
│ (BatchNormalizatio… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 64, 64,    │          0 │ activation_1[0][… │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 64, 64,    │     55,360 │ max_pooling3d[0]… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 64, 64,    │    110,656 │ activation_2[0][… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 32, 32,    │          0 │ activation_3[0][… │
│ (MaxPooling3D)      │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 32, 32,    │    221,312 │ max_pooling3d_1[… │
│                     │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv3d_4[0][0]  

 Total params: 5,608,036 (21.39 MB)

 Trainable params: 5,605,220 (21.38 MB)

 Non-trainable params: 2,816 (11.00 KB)

In [ ]:
steps_per_epoch = 100
validation_steps = 50

history = model_unet.fit(
    train_ds,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=callbacks)

In [ ]:
plt.figure()
plt.plot(history.history['dice_coef'], label='Dice - train')
plt.plot(history.history['val_dice_coef'], label='Dice - val')
plt.xlabel('Epoch')
plt.ylabel('Dice Coefficient')
plt.title('Évolution du Dice Coefficient')
plt.legend()
plt.grid(True)
plt.show()


Charger le modèle entrainé

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- (re)définir les losses utilisées pour l'entraînement ---

def sparse_dice_loss(y_true, y_pred, smooth=1e-5):
    y_true = tf.cast(y_true, tf.int32)          # très important
    y_true = tf.squeeze(y_true, axis=-1)        # (B,H,W,D)
    y_true_one_hot = tf.one_hot(
        y_true,
        depth=tf.shape(y_pred)[-1],
        dtype=tf.float32
    )

    axes = (1, 2, 3)
    intersection = tf.reduce_sum(y_true_one_hot * y_pred, axis=axes)
    denom = tf.reduce_sum(y_true_one_hot + y_pred, axis=axes)

    dice = (2. * intersection + smooth) / (denom + smooth)
    dice_loss = 1. - dice
    return tf.reduce_mean(dice_loss)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    ce = tf.reduce_mean(ce)
    dsc = sparse_dice_loss(y_true, y_pred)
    return ce + dsc

# --- Charger le modèle ---

model_path = "/content/best_unet3d.h5"

model_unet = load_model(
    model_path,
    custom_objects={
        "combined_loss": combined_loss})

model_unet.summary()

In [ ]:
PREPROC_ROOT = Path("/content/brats2023_preprocessed")

# Tous les cas
all_npz = sorted(PREPROC_ROOT.glob("*.npz"))
print("Nb de cas :", len(all_npz))

# Cas numéro 5
sample_path = all_npz[5]

print("Cas choisi :", sample_path)

data = np.load(sample_path)
img = data["image"].astype(np.float32)    # (H,W,D,4)
seg_true = data["label"].astype(np.uint8) # (H,W,D)  labels 0..3

print("img shape:", img.shape)
print("seg_true shape:", seg_true.shape)


In [ ]:
PATCH_SIZE = (128, 128, 128)

def center_crop_3d(vol, target_shape):
    """
    vol: np.ndarray de shape (H,W,D, C) ou (H,W,D)
    target_shape: (Ph, Pw, Pd)
    """
    H, W, D = vol.shape[:3]
    Ph, Pw, Pd = target_shape

    h0 = (H - Ph) // 2
    w0 = (W - Pw) // 2
    d0 = (D - Pd) // 2

    h1 = h0 + Ph
    w1 = w0 + Pw
    d1 = d0 + Pd

    if vol.ndim == 4:
        return vol[h0:h1, w0:w1, d0:d1, :]
    else:
        return vol[h0:h1, w0:w1, d0:d1]


In [ ]:
data = np.load(sample_path)
img = data["image"].astype(np.float32)      # (160,192,160,4)
seg_true = data["label"].astype(np.uint8)   # (160,192,160)

# center crop pour matcher le modèle
img_c = center_crop_3d(img, PATCH_SIZE)          # (128,128,128,4)
seg_c = center_crop_3d(seg_true, PATCH_SIZE)     # (128,128,128)

print("img_c shape:", img_c.shape)
print("seg_c shape:", seg_c.shape)

# Prédiction
input_volume = np.expand_dims(img_c, axis=0)      # (1,128,128,128,4)
pred_probs = model_unet.predict(input_volume)
pred_labels = np.argmax(pred_probs, axis=-1)[0]   # (128,128,128)

In [ ]:
import matplotlib.pyplot as plt

FLAIR_CHANNEL = 3
H, W, D, C = img_c.shape
z = D // 2

flair_slice = img_c[:, :, z, FLAIR_CHANNEL]
gt_slice = seg_c[:, :, z]
pred_slice = pred_labels[:, :, z]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(flair_slice.T, cmap="gray", origin="lower")
axes[0].set_title("FLAIR (input)")
axes[0].axis("off")

im1 = axes[1].imshow(gt_slice.T, cmap="jet", origin="lower", vmin=0, vmax=3)
axes[1].set_title("Masque GT (crop)")
axes[1].axis("off")
fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

im2 = axes[2].imshow(pred_slice.T, cmap="jet", origin="lower", vmin=0, vmax=3)
axes[2].set_title("Masque prédiction (crop)")
axes[2].axis("off")
fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


Overlay du masque

In [ ]:
# Masques masqués là où c'est background (classe 0)
gt_mask = np.ma.masked_where(gt_slice.T == 0, gt_slice.T)
pred_mask = np.ma.masked_where(pred_slice.T == 0, pred_slice.T)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(flair_slice.T, cmap="gray", origin="lower")
axes[0].imshow(gt_mask, cmap="jet", alpha=0.5, origin="lower")
axes[0].set_title("GT overlay")
axes[0].axis("off")

axes[1].imshow(flair_slice.T, cmap="gray", origin="lower")
axes[1].imshow(pred_mask, cmap("jet"), alpha=0.5, origin="lower")
axes[1].set_title("Pred overlay")
axes[1].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

# Binaire : voxels où il y a une tumeur (GT)
gt_bin = (seg_c > 0).astype(np.uint8)   # (128,128,128)

# Créer les grilles de coordonnées
x, y, z = np.mgrid[
    0:gt_bin.shape[0],
    0:gt_bin.shape[1],
    0:gt_bin.shape[2]
]

fig = go.Figure(data=go.Volume(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=gt_bin.flatten().astype(np.float32),
    isomin=0.5,        # seuil pour voir la tumeur
    isomax=1.0,
    opacity=0.1,       # transparence du volume
    surface_count=3,   # nombre de surfaces iso
))

fig.update_layout(
    title="Masque tumoral GT - volume 3D",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()


In [ ]:
# Binaire : voxels prédits comme tumeur
pred_bin = (pred_labels > 0).astype(np.uint8)

x, y, z = np.mgrid[
    0:gt_bin.shape[0],
    0:gt_bin.shape[1],
    0:gt_bin.shape[2]
]

fig = go.Figure()

# GT en bleu
fig.add_trace(go.Volume(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=gt_bin.flatten().astype(np.float32),
    isomin=0.5,
    isomax=1.0,
    opacity=0.1,
    surface_count=3,
    name="GT",
    colorscale="Blues",
    showscale=False,
))

# Pred en rouge
fig.add_trace(go.Volume(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=pred_bin.flatten().astype(np.float32),
    isomin=0.5,
    isomax=1.0,
    opacity=0.2,
    surface_count=3,
    name="Prediction",
    colorscale="Reds",
    showscale=False,
))

fig.update_layout(
    title="Masque tumoral GT (bleu) vs prédiction (rouge)",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()
